In [11]:
from bin.game_interfaces_binding import *

In [12]:
action_path = "test-cases/match-395.txt"
map_path = "test-cases/map-395-ctu-12-11-game-1.txt"

In [13]:
from utils import load_map

mapdata = load_map(map_path)
mapdata

{'game_map': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
def map_tile_to_bitmask(tile):
    if tile == 1:
        return 1 << TileMask.POND.value
    elif tile == 2:
        return 1 << TileMask.CASTLE.value
    return 0

map_formatted = [[map_tile_to_bitmask(x) for x in row] for row in mapdata['game_map']]

map_formatted

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [15]:
counter = 0
craftsman_strid_to_intid = {}
craftsmen_formatted = []
for c in mapdata['craftsmen']['team1']:
    new_craftsman = Craftsman(counter, c['x'], c['y'], True)
    craftsmen_formatted.append(new_craftsman)
    craftsman_strid_to_intid[c['id']] = counter
    counter += 1
for c in mapdata['craftsmen']['team2']:
    new_craftsman = Craftsman(counter, c['x'], c['y'], False)
    craftsmen_formatted.append(new_craftsman)
    craftsman_strid_to_intid[c['id']] = counter
    counter += 1

print(craftsman_strid_to_intid)

{'04279765': 0, '8c9874f5': 1, '723c123c': 2, '706a6acd': 3, '82600762': 4, '9c30f51c': 5, '84ffa8ce': 6, 'a2f45c95': 7, 'dbd2ea66': 8, '8e46d212': 9, '7e60979f': 10, '74f261dc': 11, '313de9f6': 12, 'c76835ff': 13, '058624ce': 14, 'a22eb972': 15, 'f0f61578': 16, 'ae942549': 17, '81939393': 18, '8061c8cb': 19}


In [16]:
actiontxt = open(action_path, "r").read()

# Split the input string based on "- -"
turns = actiontxt.strip().split("- -")


# Remove empty strings from the list
turns = [section.strip() for section in turns]
print (turns)

# Convert each section to an array of arrays
actions = [[action.split(' ') for action in (turn.split('\n') if turn else [])] for turn in turns]

idxToActionEnum = [
    [ActionType.MOVE, SubActionType.MOVE_UP],
    [ActionType.MOVE, SubActionType.MOVE_DOWN],
    [ActionType.MOVE, SubActionType.MOVE_LEFT],
    [ActionType.MOVE, SubActionType.MOVE_RIGHT],
    [ActionType.MOVE, SubActionType.MOVE_UP_LEFT],
    [ActionType.MOVE, SubActionType.MOVE_UP_RIGHT],
    [ActionType.MOVE, SubActionType.MOVE_DOWN_LEFT],
    [ActionType.MOVE, SubActionType.MOVE_DOWN_RIGHT],
    [ActionType.BUILD, SubActionType.BUILD_UP],
    [ActionType.BUILD, SubActionType.BUILD_DOWN],
    [ActionType.BUILD, SubActionType.BUILD_LEFT],
    [ActionType.BUILD, SubActionType.BUILD_RIGHT],
    [ActionType.DESTROY, SubActionType.DESTROY_UP],
    [ActionType.DESTROY, SubActionType.DESTROY_DOWN],
    [ActionType.DESTROY, SubActionType.DESTROY_LEFT],
    [ActionType.DESTROY, SubActionType.DESTROY_RIGHT],
    [ActionType.STAY, SubActionType.STAY],
  ]

actions = [[GameAction(craftsman_strid_to_intid[action[0]], *idxToActionEnum[int(action[1])]) for action in turn] for turn in actions]

['04279765 10\n8c9874f5 9\n723c123c 9\n706a6acd 9\n82600762 9\n9c30f51c 9\n84ffa8ce 9\na2f45c95 9\ndbd2ea66 9\n8e46d212 9', '7e60979f 8\n74f261dc 8\n313de9f6 8\nc76835ff 8\n058624ce 8\na22eb972 8\nf0f61578 8\nae942549 8\n81939393 8\n8061c8cb 8', '04279765 10\n8c9874f5 9\n723c123c 9\n706a6acd 9\n82600762 9\n9c30f51c 9\n84ffa8ce 9\na2f45c95 9\ndbd2ea66 9\n8e46d212 9', '7e60979f 9\n74f261dc 9\n313de9f6 9\nc76835ff 9\n058624ce 9\na22eb972 9\nf0f61578 9\nae942549 9\n81939393 9\n8061c8cb 9', '04279765 4\n8c9874f5 10\n723c123c 10\n706a6acd 10\n82600762 10\n9c30f51c 11\n84ffa8ce 10\na2f45c95 10\ndbd2ea66 10\n8e46d212 10', '7e60979f 10\n74f261dc 10\n313de9f6 10\nc76835ff 10\n058624ce 10\na22eb972 11\nf0f61578 10\nae942549 11\n81939393 10\n8061c8cb 4', '04279765 8\n8c9874f5 11\n723c123c 11\n706a6acd 11\n82600762 11\n9c30f51c 10\n84ffa8ce 11\na2f45c95 11\ndbd2ea66 11\n8e46d212 11', '', '04279765 10\n8c9874f5 4\n723c123c 4\n706a6acd 5\n82600762 4\n9c30f51c 4\n84ffa8ce 4\na2f45c95 4\ndbd2ea66 4\n8e

In [17]:
go = GameOptions()
go.mapWidth = mapdata['game_settings']['map_width']
go.mapHeight = mapdata['game_settings']['map_height']
go.maxTurns =  mapdata['game_settings']['max_turn']
go.wallCoeff =  mapdata['score_coefficients']['wall']
go.castleCoeff =   mapdata['score_coefficients']['castle']
go.territoryCoeff =   mapdata['score_coefficients']['territory']

game = Game(
    go,
    map_formatted,
    craftsmen_formatted
)

In [18]:
for i, turn in enumerate(actions):
    for action in turn:
        game.addAction(action)
    game.nextTurn()
    print(f'Turn {i+2}:',game.getCurrentState().map.calcPoints(game.gameOptions, True), game.getCurrentState().map.calcPoints(game.gameOptions, False))
    # for action in game.getCurrentState().lastTurnActions:
    #     print(action.craftsmanId, action.actionType, action.subActionType)
    print(game.getCurrentState().map.printMap())
    print()


print(game.getCurrentState().map.calcPoints(game.gameOptions, True), game.getCurrentState().map.calcPoints(game.gameOptions, False))

Turn 2: 10 0
0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 
0000000000 0000000000 0000000000 0000000000 0000100000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000001 0000100000 0000000000 0000010000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 
0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000001 0000000000 0000000000 0000000000 0000000000 0000000000 0000000100 0000000000 0000100000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 
0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 0000000000 00000

In [19]:
import threading
import time

In [20]:
start = time.time()

def run_game():
    for i in range(100):
        go = GameOptions()
        go.mapWidth = mapdata['game_settings']['map_width']
        go.mapHeight = mapdata['game_settings']['map_height']
        go.maxTurns =  mapdata['game_settings']['max_turn']
        go.wallCoeff =  mapdata['score_coefficients']['wall']
        go.castleCoeff =   mapdata['score_coefficients']['castle']
        go.territoryCoeff =   mapdata['score_coefficients']['territory']

        game = Game(
            go,
            map_formatted,
            craftsmen_formatted
        )

        for turn in actions:
            for action in turn:
                game.addAction(action)
            game.nextTurn()
    # print(time.time() - start)

threads = []
for i in range(24):
    t = threading.Thread(target=run_game)
    t.start()
    threads.append(t)

for t in threads:
    t.join()
print(time.time() - start)

17.685230016708374
